# DPPUv2 paper02: Paper Figures
## Structural Robustness of Isotropic S³ Vacua in Einstein-Cartan Minisuperspace via Chiral Equilibrium and Weyl Stability

This notebook generates the key figures for the DPPUv2 paper02 regarding the analysis (Weyl anisotropy).
It performs the following steps:
1.  **Data Generation**: Executes the core analysis scripts (`potential_landscape_mapping`, `critical_analysis`, `topology_comparison`) directly to produce raw data.
2.  **Visualization**: Plots the results to demonstrate the stability mechanism, bifurcation structure, and topological preference.

**Author**: Muacca
**Date**: 2026-02-19



In [ ]:
import sys
import os
import importlib.util
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Set plotting style
sns.set_theme(style="whitegrid")
plt.rcParams.update({
    'font.size': 14,
    'axes.titlesize': 16,
    'axes.labelsize': 14,
    'xtick.labelsize': 12,
    'ytick.labelsize': 12,
    'legend.fontsize': 12,
    'figure.figsize': (10, 6)
})

# Path Setup
# This notebook lives in scripts/paper02/, so project root is 2 levels up
notebook_dir = os.path.dirname(os.path.abspath("__file__"))
project_root = os.path.normpath(os.path.join(notebook_dir, "..", ".."))
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    print(f"Added to path: {project_root}")

# Output directory for data generated by this notebook
output_dir = os.path.join(project_root, "output", "paper02")
os.makedirs(output_dir, exist_ok=True)
print(f"Data output directory: {output_dir}")

## 1. Import Analysis Modules
We import the analysis scripts dynamically from the source repository to ensure reproducibility.


In [ ]:
def import_module_from_path(module_name, file_path):
    spec = importlib.util.spec_from_file_location(module_name, file_path)
    if spec is None:
        raise ImportError(f"Could not load spec for {module_name} from {file_path}")
    module = importlib.util.module_from_spec(spec)
    sys.modules[module_name] = module
    spec.loader.exec_module(module)
    return module

scripts_dir = os.path.join(project_root, "scripts", "paper02")

# Import potential_landscape_mapping.py
mapping = import_module_from_path("mapping", os.path.join(scripts_dir, "potential_landscape_mapping.py"))

# Import critical_analysis.py
critical = import_module_from_path("critical", os.path.join(scripts_dir, "critical_analysis.py"))

# Import topology_comparison.py
comparison = import_module_from_path("comparison", os.path.join(scripts_dir, "topology_comparison.py"))

print("Successfully imported analysis modules.")

## 2. Potential Landscapes (Fig 2)
Mapping the effective potential $V_{eff}(r, \epsilon)$ for S3 topology at different Weyl couplings $\alpha$.
We examine $\alpha = -10$ (stable), $\alpha = 0$ (marginal/boundary), and $\alpha = 10$ (unstable).


In [ ]:
# Generate Mapping Data (always computed from engine)
alphas = [-10, 0, 10]
topology = "S3"
mapping_files = {}

for alpha in alphas:
    fname = f"mapping_{topology}_alpha{alpha}.csv"
    fpath = os.path.join(output_dir, fname)
    mapping_files[alpha] = fpath
    
    print(f"Generating mapping for alpha={alpha}...")
    mapping.run_mapping(
        topology=topology,
        alpha_val=alpha,
        r_range=(0.01, 5.0),
        eps_range=(-0.9, 0.9),
        resolution=100,
        output_file=fpath
    )

print("All mapping data generated.")


In [ ]:
# Plot Fig 2: Potential Landscapes
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

levels = np.linspace(-10, 10, 21) # Define contour levels

for i, alpha in enumerate(alphas):
    df = pd.read_csv(mapping_files[alpha])
    
    # Reshape for contour plot
    # Assuming the data was generated on a grid, we need to determine grid size
    n_unique = int(np.sqrt(len(df)))
    R = df['r'].values.reshape(n_unique, n_unique)
    Eps = df['epsilon'].values.reshape(n_unique, n_unique)
    V = df['V_eff'].values.reshape(n_unique, n_unique)
    
    ax = axes[i]
    # Log scale for potential might be tricky with negative values, checking range
    # Just use linear scale with clamped range for visibility
    
    cf = ax.contourf(R, Eps, V, levels=50, cmap='RdBu_r', vmin=-100, vmax=100, extend='both')
    ax.set_title(f"$\\alpha = {alpha}$")
    ax.set_xlabel("$r$")
    if i == 0:
        ax.set_ylabel("$\\epsilon$")
    
    # Mark the minimum if it exists and is within range
    # Simple check for min in data
    min_idx = df['V_eff'].idxmin()
    min_row = df.loc[min_idx]
    if min_row['V_eff'] > -1e9: # Arbitrary large negative cutoff for instability
        ax.plot(min_row['r'], min_row['epsilon'], 'y*', markersize=12, markeredgecolor='black', label='Min')

fig.colorbar(cf, ax=axes.ravel().tolist(), label="$V_{eff}$")
plt.suptitle(f"S3 Potential Landscape Evolution ($\\alpha$ dependence)", fontsize=16)
plt.savefig(os.path.join(output_dir, "fig02_landscape.png"), dpi=300, bbox_inches='tight')
plt.show()



## 3. Stability Bifurcation (Fig 3 & 4)
Analyzing the critical behavior of the order parameter $\epsilon_*$ and size modulus $r_*$ as $\alpha$ crosses 0.
We perform a fine-grained scan near $\alpha=0$ and a broader scan.


In [ ]:
# Generate Critical Analysis Data (always computed from engine)
# 1. Broad Scan
broad_file = os.path.join(output_dir, "critical_S3_broad.csv")
print("Running broad critical analysis...")
critical.analyze_critical_behavior("S3", (-1.0, 1.0), 0.05, broad_file)

# 2. Fine Scan
fine_file = os.path.join(output_dir, "critical_S3_fine.csv")
print("Running fine critical analysis...")
critical.analyze_critical_behavior("S3", (-0.15, 0.15), 0.005, fine_file)

print("Critical analysis complete.")


In [ ]:
# Plot Fig 3: Order Parameter Bifurcation (Epsilon vs Alpha)
# Two-panel layout: Left = broad scan, Right = fine scan (inset-style zoom)
df_fine = pd.read_csv(fine_file)
df_broad = pd.read_csv(broad_file)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5), gridspec_kw={'width_ratios': [1.2, 1]})

# Left panel: Broad scan (alpha in [-1, 1])
ax1.plot(df_broad['alpha'], df_broad['epsilon_star'], 'o-', color='steelblue', markersize=3)
ax1.set_xlabel(r"Weyl Coupling $\alpha$")
ax1.set_ylabel(r"Order Parameter $\epsilon_*$")
ax1.set_title(r"(a) Broad Scan: $\alpha \in [-1, 1]$")
ax1.axvline(0, color='k', linestyle='--', alpha=0.3, label=r"$\alpha=0$")
ax1.legend()
ax1.grid(True)

# Right panel: Fine scan near alpha=0 (inset-style zoom)
ax2.plot(df_fine['alpha'], df_fine['epsilon_star'], 's-', color='crimson', markersize=4)
ax2.set_xlabel(r"Weyl Coupling $\alpha$")
ax2.set_ylabel(r"Order Parameter $\epsilon_*$")
ax2.set_title(r"(b) Fine Scan: $\alpha \in [-0.15, 0.15]$")
ax2.axvline(0, color='k', linestyle='--', alpha=0.3, label=r"$\alpha=0$")
ax2.legend()
ax2.grid(True)

plt.suptitle(r"Stability Bifurcation ($\epsilon_*$ vs $\alpha$, $S^3$)", fontsize=16, y=1.02)
plt.tight_layout()
plt.savefig(os.path.join(output_dir, "fig03_bifurcation.png"), dpi=300, bbox_inches='tight')
plt.show()


## 4. Size-Shape Decoupling (Fig 4)
Verifying that the size modulus $r_*$ remains stable (finite) even as $\epsilon_*$ changes, for $\alpha \le 0$.


In [ ]:
# Plot Fig 4: Size Modulus Stability (r vs Alpha)
plt.figure(figsize=(8, 6))

# Filter bounded results for plotting r
mask_fine = df_fine['r_star'] < 20
mask_broad = df_broad['r_star'] < 20

plt.plot(df_broad[mask_broad]['alpha'], df_broad[mask_broad]['r_star'], 'o-', label='Broad Scan', alpha=0.5)
plt.plot(df_fine[mask_fine]['alpha'], df_fine[mask_fine]['r_star'], '*-', label='Fine Scan', color='r')

plt.xlabel(r"Weyl Coupling $\alpha$")
plt.ylabel(r"Size Modulus $r_*$")
plt.title(r"Size Modulus Stability ($r_*$ vs $\alpha$)")
plt.axvline(0, color='k', linestyle='--', alpha=0.3)
plt.legend()
plt.grid(True)
plt.savefig(os.path.join(output_dir, "fig04_size_stability.png"), dpi=300)
plt.show()



## 5. Topology Comparison (Fig 5)
Comparing the vacuum energy $V_{min}$ across S3, T3, and Nil3 topologies to determine the preferred ground state.


In [ ]:
# Generate Comparison Data (always computed from engine)
comp_file = os.path.join(output_dir, "topology_comparison.csv")
print("Running topology comparison...")
alphas_comp = np.linspace(-5, 5, 21)
comparison.run_comparison(alphas_comp, comp_file)

print("Topology comparison complete.")


In [ ]:
# Plot Fig 5: Topology Energy Comparison
df_comp = pd.read_csv(comp_file)

plt.figure(figsize=(10, 6))
plt.plot(df_comp['alpha'], df_comp['V_min_S3'], 'o-', label='S3')
plt.plot(df_comp['alpha'], df_comp['V_min_T3'], 's--', label='T3')
plt.plot(df_comp['alpha'], df_comp['V_min_Nil3'], '^-.', label='Nil3')

plt.xlabel(r"$\alpha$")
plt.ylabel(r"Minimum Potential $V_{min}$")
plt.title("Vacuum Energy Comparison (S3 vs T3 vs Nil3)")
plt.ylim(-1000, 100) # Clamp y-axis to see the details near zero, ignoring deep instability
plt.legend()
plt.grid(True)
plt.savefig(os.path.join(output_dir, "fig05_topology_comparison.png"), dpi=300)
plt.show()



## 6. Parameter Dependence (Fig 6)
Verifying that the stability boundary at $\alpha=0$ is independent of paper01 parameters $(V, \eta, \theta_{NY})$.
We scan 4 representative parameter sets across the paper01 landscape:
1. **Type I center**: $\eta = -3.0$ (deep stable vacuum)
2. **I/II Boundary**: $\eta = -2.0$ (reference parameter set)
3. **Type II center**: $\eta = 0.0$ (no axial torsion)
4. **II/III Boundary**: $\eta = 2.0$ (marginal stability)

All sets use $V=4, \theta_{NY}=1, \kappa=1, L=1$.


In [ ]:
# Generate scan data from engine (zero-base computation)
# 4 parameter sets spanning the paper01 landscape
gamma2_params = [
    ("Type I",       {'V': 4.0, 'eta': -3.0, 'theta_NY': 1.0, 'L': 1.0, 'kappa': 1.0}),
    ("I/II Boundary", {'V': 4.0, 'eta': -2.0, 'theta_NY': 1.0, 'L': 1.0, 'kappa': 1.0}),
    ("Type II",      {'V': 4.0, 'eta':  0.0, 'theta_NY': 1.0, 'L': 1.0, 'kappa': 1.0}),
    ("II/III Boundary", {'V': 4.0, 'eta':  2.0, 'theta_NY': 1.0, 'L': 1.0, 'kappa': 1.0}),
]

gamma2_sets = {}  # label -> filepath

for name, params in gamma2_params:
    fname = f"gamma2_{name.replace(' ', '_').replace('/', '_')}.csv"
    fpath = os.path.join(output_dir, fname)
    label = f"{name} ($\\eta={params['eta']:.1f}$)"
    gamma2_sets[label] = fpath
    
    print(f"Running scan: {name} (η={params['eta']})...")
    critical.analyze_critical_behavior(
        topology="S3",
        alpha_range=(-1.0, 1.0),
        alpha_step=0.01,
        output_file=fpath,
        phase1_params=params
    )

print("All scans complete.")


In [ ]:
# Plot Fig 6: Parameter Dependence — 2×2 Facet Grid of V_min(α)
fig, axes = plt.subplots(2, 2, figsize=(14, 10), sharex=True)
axes_flat = axes.ravel()

colors = ['steelblue', 'forestgreen', 'darkorange', 'crimson']
eta_values = [-3.0, -2.0, 0.0, 2.0]

for i, (label, fpath) in enumerate(gamma2_sets.items()):
    ax = axes_flat[i]
    df_g = pd.read_csv(fpath)
    
    ax.plot(df_g['alpha'], df_g['V_min'], 'o-', color=colors[i], markersize=2, linewidth=1.2)
    ax.axvline(0, color='k', linestyle='--', alpha=0.4, linewidth=0.8)
    ax.set_title(label, fontsize=13)
    ax.set_ylabel(r"$V_{\min}$")
    ax.grid(True, alpha=0.3)
    
    # Annotate V_min value in the stable region (alpha <= 0)
    stable = df_g[df_g['alpha'] <= 0]
    v_stable = stable['V_min'].iloc[0]
    ax.annotate(f"$V_{{\\min}} = {v_stable:.1f}$",
                xy=(stable['alpha'].iloc[0], v_stable),
                xytext=(0.05, 0.85), textcoords='axes fraction',
                fontsize=10, color=colors[i],
                bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.8))

# Set x-labels on bottom row only
for ax in axes[1]:
    ax.set_xlabel(r"Weyl Coupling $\alpha$")

plt.suptitle(r"Parameter Dependence — $V_{\min}(\alpha)$ across Phase 1 Parameters",
             fontsize=15, y=1.01)
plt.tight_layout()
plt.savefig(os.path.join(output_dir, "fig06_gamma2_facet.png"), dpi=300, bbox_inches='tight')
plt.show()


## 7. T³ Null Test (Fig 7)
$T^3$ is flat ($C^2 = 0$), so the Weyl term contributes nothing. This provides a strict null test:
$r_*$, $\epsilon_*$, and $V_{\min}$ must all be independent of $\alpha$.

Data: 201 points, $\alpha \in [-1, 1]$, step 0.01.

3-panel layout: $r_*(\alpha)$, $\epsilon_*(\alpha)$, $V_{\min}(\alpha)$.


In [ ]:
# Generate T3 null test data from engine (zero-base computation)
t3_file = os.path.join(output_dir, "critical_T3_nulltest.csv")

print("Running T³ null test (α ∈ [-1, 1], step 0.01)...")
critical.analyze_critical_behavior("T3", (-1.0, 1.0), 0.01, t3_file)

df_t3 = pd.read_csv(t3_file)
print(f"T3 null test complete: {len(df_t3)} points")
print(f"  r* range:  [{df_t3['r_star'].min():.10f}, {df_t3['r_star'].max():.10f}]")
print(f"  ε* range:  [{df_t3['epsilon_star'].min():.10f}, {df_t3['epsilon_star'].max():.10f}]")
print(f"  Vmin range: [{df_t3['V_min'].min():.4e}, {df_t3['V_min'].max():.4e}]")


In [ ]:
# Plot Fig 7: T³ Null Test — 3-panel visualization
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 5), sharex=True)

color_t3 = 'teal'

# Panel (a): r*(α)
ax1.plot(df_t3['alpha'], df_t3['r_star'], '-', color=color_t3, linewidth=1.5)
ax1.set_ylabel(r"$r_*$")
ax1.set_title(r"(a) Size Modulus $r_*(\alpha)$")
ax1.set_xlabel(r"$\alpha$")
ax1.grid(True, alpha=0.3)
# Add reference line at the constant value
r_mean = df_t3['r_star'].mean()
ax1.axhline(r_mean, color='gray', linestyle=':', alpha=0.5)
ax1.annotate(f"$r_* = {r_mean:.3f}$", xy=(0.05, 0.9), xycoords='axes fraction',
             fontsize=11, bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.8))

# Panel (b): ε*(α)
ax2.plot(df_t3['alpha'], df_t3['epsilon_star'], '-', color=color_t3, linewidth=1.5)
ax2.set_ylabel(r"$\epsilon_*$")
ax2.set_title(r"(b) Anisotropy $\epsilon_*(\alpha)$")
ax2.set_xlabel(r"$\alpha$")
ax2.grid(True, alpha=0.3)
eps_mean = df_t3['epsilon_star'].mean()
ax2.axhline(eps_mean, color='gray', linestyle=':', alpha=0.5)
ax2.annotate(f"$\\epsilon_* = {eps_mean:.3f}$", xy=(0.05, 0.9), xycoords='axes fraction',
             fontsize=11, bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.8))

# Panel (c): V_min(α)
ax3.plot(df_t3['alpha'], df_t3['V_min'], '-', color=color_t3, linewidth=1.5)
ax3.set_ylabel(r"$V_{\min}$")
ax3.set_title(r"(c) Vacuum Energy $V_{\min}(\alpha)$")
ax3.set_xlabel(r"$\alpha$")
ax3.grid(True, alpha=0.3)
v_mean = df_t3['V_min'].mean()
ax3.axhline(v_mean, color='gray', linestyle=':', alpha=0.5)
ax3.annotate(f"$V_{{\\min}} = {v_mean:.2e}$", xy=(0.05, 0.9), xycoords='axes fraction',
             fontsize=11, bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.8))

plt.suptitle(r"$T^3$ Null Test — All quantities $\alpha$-independent ($C^2 = 0$)",
             fontsize=15, y=1.02)
plt.tight_layout()
plt.savefig(os.path.join(output_dir, "fig07_t3_null_test.png"), dpi=300, bbox_inches='tight')
plt.show()

# Quantitative null test summary
print("\n=== T³ Null Test Summary ===")
print(f"  max|r* - <r*>|   = {(df_t3['r_star'] - r_mean).abs().max():.2e}")
print(f"  max|ε* - <ε*>|   = {(df_t3['epsilon_star'] - eps_mean).abs().max():.2e}")
print(f"  max|Vmin - <Vmin>| = {(df_t3['V_min'] - v_mean).abs().max():.2e}")
print(f"  → All deviations at machine precision: NULL TEST PASSED")


## Conclusion
The figures generated above confirm the robust stability mechanism of the DPPUv2 model:
1.  **Fig 2 — Potential Landscape**: The effective potential $V_{eff}(r, \epsilon)$ shows clear vacuum structure for $\alpha \le 0$ and unbounded instability for $\alpha > 0$.
2.  **Fig 3 — Bifurcation**: A sharp transition in $\epsilon_*$ occurs exactly at $\alpha=0$, confirmed by both broad and fine scans.
3.  **Fig 4 — Size Stability**: The size modulus $r_*$ remains finite and stable for $\alpha \le 0$.
4.  **Fig 5 — Topology Comparison**: $S^3$ is energetically preferred over $T^3$ and $Nil^3$ in the stable regime.
5.  **Fig 6 — Parameter Universality**: The $\alpha=0$ stability boundary is independent of Phase 1 parameters $(V, \eta, \theta_{NY})$, verified across 4 representative parameter sets.
6.  **Fig 7 — $T^3$ Null Test**: All quantities ($r_*$, $\epsilon_*$, $V_{\min}$) are $\alpha$-independent for $T^3$ ($C^2=0$), confirming engine correctness at machine precision.
